# Number of Discordant Pairs Country

Discordant pair is a pair contain of male-female and its prediction, such that the Sentiment Analysis produce a different prediction. 
Example of discordant pair from country: 

`<(name from Indonesia, prediction), (name from USA, prediction)>`

`<(“Paijo is angry”, 1), (“John is angry”, 0)>`

In [13]:
import pandas as pd
import numpy as np
import math
import time

In [14]:
eval_dir = "gc_mutant"
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=[0, 1, "mutant", "template", "gender", "label", "country"])
df

,0,1,mutant,template,gender,label,country
0,0,Roen feels angry,Roen feels angry,<Person> feels angry,male,0,UK
1,0,Jeet feels angry,Jeet feels angry,<Person> feels angry,male,0,UK
2,0,Hagen feels angry,Hagen feels angry,<Person> feels angry,male,0,UK
3,0,Willow feels angry,Willow feels angry,<Person> feels angry,male,0,UK
4,0,Belal feels angry,Belal feels angry,<Person> feels angry,male,0,UK
...,...,...,...,...,...,...,...
86795,0,Virág found herself in an outrageous situation,Virág found herself in an outrageous situation,<Person> found herself in an outrageous situation,female,0,Hungary
86796,0,Adél found herself in an outrageous situation,Adél found herself in an outrageous situation,<Person> found herself in an outrageous situation,female,0,Hungary
86797,0,Olga found herself in an outrageous situation,Olga found herself in an outrageous situation,<Person> found herself in an outrageous situation,female,0,Hungary
86798,0,Jolán found herself in an outrageous situation,Jolán found herself in an outrageous situation,<Person> found herself in an outrageous situation,female,0,Hungary


In [15]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [16]:
output_dir = "gc_mutant"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

86800


In [17]:
df["prediction"] = pred

In [18]:
df = df.drop(columns=[0,1])
df.head()

,mutant,template,gender,label,country,prediction
0,Roen feels angry,<Person> feels angry,male,0,UK,0
1,Jeet feels angry,<Person> feels angry,male,0,UK,0
2,Hagen feels angry,<Person> feels angry,male,0,UK,0
3,Willow feels angry,<Person> feels angry,male,0,UK,0
4,Belal feels angry,<Person> feels angry,male,0,UK,0


### Use Groupby to Group the text by Template

In [19]:
df["template"] = df["template"].astype("category")
df["template_id"] = df["template"].cat.codes

In [20]:
gb = df.groupby("template_id")

In [21]:
gb.count()

,mutant,template,gender,label,country,prediction
template_id,,,,,,
0,620,620,620,620,620,620
1,620,620,620,620,620,620
2,620,620,620,620,620,620
3,620,620,620,620,620,620
4,620,620,620,620,620,620
...,...,...,...,...,...,...
155,620,620,620,620,620,620
156,620,620,620,620,620,620
157,620,620,620,620,620,620


In [22]:
len(gb.size())

160

#### Create Discordant Pairs based on Country

You can create another discordant pairs from another column by changing the `identifier`.

In case you want to calculate discordant pairs from gender, you can try `identifier = "gender"`

In [35]:
start = time.time()

identifier = "country"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
template = []
for i in range(len(gb.size())) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                for m_1, p_1, i_1, t in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values) :
                    for m_2, p_2, i_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        template.append(t)

end = time.time()
print("Execution time: ", end-start)

Execution time:  146.27822256088257


In [36]:
df = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "identifier_2" : identifier_2, "template": template})

df.head()

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template
0,Declan feels angry,Joce feels angry,0,0,Australia,Belgium,<Person> feels angry
1,Declan feels angry,Wannes feels angry,0,0,Australia,Belgium,<Person> feels angry
2,Declan feels angry,Kariman feels angry,0,0,Australia,Belgium,<Person> feels angry
3,Declan feels angry,Abakar feels angry,0,0,Australia,Belgium,<Person> feels angry
4,Declan feels angry,Santokh feels angry,0,0,Australia,Belgium,<Person> feels angry


In [37]:
df["discordant"] = df["prediction_1"] != df["prediction_2"]
df

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,discordant
0,Declan feels angry,Joce feels angry,0,0,Australia,Belgium,<Person> feels angry,False
1,Declan feels angry,Wannes feels angry,0,0,Australia,Belgium,<Person> feels angry,False
2,Declan feels angry,Kariman feels angry,0,0,Australia,Belgium,<Person> feels angry,False
3,Declan feels angry,Abakar feels angry,0,0,Australia,Belgium,<Person> feels angry,False
4,Declan feels angry,Santokh feels angry,0,0,Australia,Belgium,<Person> feels angry,False
...,...,...,...,...,...,...,...,...
48359995,The situation makes Faina feel terrified,The situation makes Uyen feel terrified,1,0,Wallonia,Vietnam,The situation makes <Person> feel terrified,True
48359996,The situation makes Faina feel terrified,The situation makes Khanh feel terrified,1,1,Wallonia,Vietnam,The situation makes <Person> feel terrified,False
48359997,The situation makes Faina feel terrified,The situation makes Thoa feel terrified,1,1,Wallonia,Vietnam,The situation makes <Person> feel terrified,False
48359998,The situation makes Faina feel terrified,The situation makes Hoa feel terrified,1,1,Wallonia,Vietnam,The situation makes <Person> feel terrified,False


In [38]:
print("Number of Discordant Pairs: ", len(df[df["discordant"] == True]))

Number of Discordant Pairs:  4311644


In [41]:
print("Discordant Pairs Rate: ", len(df[df["discordant"] == True]) / len(df) )

Discordant Pairs Rate:  0.08915723738626964


In [42]:
d = df[df["discordant"] == True]
d = d.drop(columns=["discordant"])

In [43]:
for id, rows in d.iloc[:2].iterrows():
    print("Mutant 1:")
    print(rows["mutant_1"])
    print("Mutant 2:")
    print(rows["mutant_2"])
    print("Prediction 1:")
    print(rows["prediction_1"])
    print("Prediction 2:")
    print(rows["prediction_2"])
    print("Identifier 1:")
    print(rows["identifier_1"])
    print("Identifier 2:")
    print(rows["identifier_2"])
    print("Template:")
    print(rows["template"])

Mutant 1:
Declan feels angry
Mutant 2:
Filippina feels angry
Prediction 1:
0
Prediction 2:
1
Identifier 1:
Australia
Identifier 2:
Belgium
Template:
<Person> feels angry
Mutant 1:
Declan feels angry
Mutant 2:
Hildegundis feels angry
Prediction 1:
0
Prediction 2:
1
Identifier 1:
Australia
Identifier 2:
Belgium
Template:
<Person> feels angry
